<a href="https://colab.research.google.com/github/khalil-alexander/get_forecast/blob/main/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astropy

In [ ]:
from astropy import coordinates as coord
from astropy import units as u
from astropy import time
from astropy.time import Time
from astropy.coordinates import Angle

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as reqs
import xml.etree.ElementTree as ET
import io
import pprint
import json
import datetime

In [ ]:
from datetime import datetime
import datetime as dt
from datetime import timezone

In [ ]:
import xarray as xr

In [ ]:
now = Time("2024-03-07 00:46:00.000", scale='utc')
xyz = [1155.746046202530, -6632.420367726780, 953.533229633281]
cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)



gcrs = coord.GCRS(cartrep, obstime = now)
itrs = gcrs.transform_to(coord.ITRS(obstime = now))
loc = coord.EarthLocation(*itrs.cartesian.xyz)

print(loc.lat, loc.lon, loc.height)

8d07m57.90103915s 103d26m20.85144985s 421.8445702007271 km


In [ ]:
col_name = ['Date','x units(km)','y units(km)','z units(km)','x_dot(km/s)','y_dot(km/s)','z_dot(km/s)']
# Challenge to have the columns read under beautiful soup
xml_df = pd.DataFrame(columns=col_name)
xml_df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)


In [ ]:
def create_latlon_positions(date: str, xyz: list[float]):
    now = Time(date, scale='utc')
    cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)

    gcrs = coord.GCRS(cartrep, obstime = now)
    itrs = gcrs.transform_to(coord.ITRS(obstime = now))
    loc = coord.EarthLocation(*itrs.cartesian.xyz)

    return Angle(loc.lat).deg, Angle(loc.lon).deg


In [ ]:
def get_latlon(row: pd.Series):
    date = row['Date'].strftime('%Y-%m-%d %H:%M:%S')
    xyz = [row['x units(km)'], row['y units(km)'], row['z units(km)']]
    return create_latlon_positions(date, xyz)


In [ ]:
def read_xml_DataFrame(url):
    """
    Do web scraping from an XML file to obtain time the International Space
    Station will be at a specfic postion and the speed it is going at.

    Parameters
    ----------
    url : String
        The url of the data in XML format you want to extract from.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station is at
        a specfic position and the speed it is traveling.
    """
    # Get the xml request
    xml = reqs.get(url)

    # Use BeautifulSoup to Organize the XML file
    soup = BeautifulSoup(xml.content,"lxml")

    # Finds all the instances 'statevector' appeared in the xml file
    state_tag = soup.find_all('statevector')

    col_name = ['Date','x units(km)',
                'y units(km)','z units(km)',
                'x_dot(km/s)',
                'y_dot(km/s)','z_dot(km/s)']
    df = pd.DataFrame(columns=col_name)

    #Loops thorugh all elements to add rows in the empty xml_df
    for element in state_tag:
        # Turns element into text and splits the '\n'
        element = element.text.split('\n')
        # Deletes First empty String
        element.pop(0)
        # Deletes Last empty Sting
        element.pop(-1)
        # Removes the 'Z' in the elements first index
        element[0] = element[0][:-1]

        # Appending list into DataFrame by adding 1 row to the DataFrame
        #df = xml_df.append(pd.DataFrame(element, columns=col_name), ignore_index=True)
        df.loc[len(df)] = element


    # Turns the "Date" collumn into a Datetime object
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%jT%H:%M:%S.%f')
    # Sets the "Date" collumn into a Datetime object
    #df.set_index("Date", inplace=True)

    # Returns the Dataframe
    return df

In [ ]:
xml_url = "https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.xml"
%time df = read_xml_DataFrame(xml_url)

<ipython-input-10-ea97148a76b9>:21: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml.content,"lxml")


CPU times: user 10.3 s, sys: 133 ms, total: 10.4 s
Wall time: 11.3 s


In [ ]:
df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)
0,2024-12-09 12:00:00,-1652.7463421948501,4395.8815000702998,-4909.1717348923903,-7.2908956842462498,-0.13615858805302999,2.33733423133949
1,2024-12-09 12:04:00,-3321.2944418473899,4203.94886381795,-4176.2363511959002,-6.5286961852710599,-1.4538256986054301,3.7334189886023199
2,2024-12-09 12:08:00,-4748.3383115019897,3706.3747371425902,-3138.8149190019199,-5.2903993132728697,-2.6675993424939999,4.8590838224203496
3,2024-12-09 12:12:00,-5829.6339441926402,2938.9506556475399,-1872.2215759620899,-3.6649357567155598,-3.6887050840348898,5.6312824230514904
4,2024-12-09 12:16:00,-6485.93127828048,1957.2855852759401,-468.76765373474802,-1.7704679716077301,-4.4418526329088497,5.9924023519317204
...,...,...,...,...,...,...,...
5931,2024-12-24 11:45:00,-2772.48102148318,-4936.5256744617,3754.4788360662701,3.5659744152822501,-5.2649704092882796,-4.2724466650143196
5932,2024-12-24 11:49:00,-1826.22081425897,-6005.23861306049,2604.66498165543,4.27139828167146,-3.5865547444380499,-5.25069125904401
5933,2024-12-24 11:53:00,-747.12417128956895,-6637.1429290016404,1264.8580507593099,4.6661571052558104,-1.6471887948936099,-5.8459904046046596
5934,2024-12-24 11:57:00,386.31372604792398,-6786.3248725805697,-167.20734433483599,4.7214973206748203,0.41143151775853998,-6.0148175475239896


In [ ]:
%time df[['lat','lon']] = df.apply(get_latlon, axis=1, result_type='expand')

CPU times: user 58.6 s, sys: 1.03 s, total: 59.6 s
Wall time: 1min


In [ ]:
df = df[['Date','lat','lon']]

In [ ]:
def obtain_future_date(df: pd.DataFrame, nhours: int=3):
    """
    Collects future position of the ISS with nhours from current time and 6
    hours ahead of nhours

    Parameters
    ----------
    df : Pandas Dataframe
        Dataframe which contains a column with datetime objects.

    nhours: Int
        Number of hours you want to start collecting data from current time.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station
        from current time + nhours to 6 hours ahead.
    """
    # String of the current time of day now
    start_date = dt.datetime.now() + dt.timedelta(hours=nhours)
    #end_date = start_date + dt.timedelta(hours=24)
    # Turns the date from a string object to a datetime object
    start_date = start_date.strftime("%m/%d/%Y %H:%M:%S")
    start_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')

    end_date = start_date + dt.timedelta(hours=6)
    #end_date = end_date.strftime("%m/%d/%Y %H:%M:%S")
    #end_date = pd.to_datetime(start_date, format='%m/%d/%Y %H:%M:%S')
    #return (start_date, end_date)

    return df[(start_date <= df['Date']) & (end_date >= df['Date'])]

In [ ]:
df1 = obtain_future_date(df)
df1.index = range(len(df1))

In [ ]:
df1

,Date,lat,lon
0,2024-12-11 20:36:00,26.958835,-78.510249
1,2024-12-11 20:40:00,15.356222,-68.410381
2,2024-12-11 20:44:00,3.251140,-59.505702
3,2024-12-11 20:48:00,-8.948911,-50.839524
4,2024-12-11 20:52:00,-20.868778,-41.513600
...,...,...,...
85,2024-12-12 02:16:00,18.365540,53.836089
86,2024-12-12 02:20:00,29.782693,64.446255
87,2024-12-12 02:24:00,39.914133,77.839800
88,2024-12-12 02:28:00,47.691038,95.498677


In [ ]:
def get_orbit(df: pd.DataFrame):
    count = 0
    temp_df = df['lon']

    for i in range(len(temp_df)-1):
        if (temp_df[i] > 0) and (temp_df[i+1] < 0):
            count += 1
            if count == 2:
                new_df = df.loc[low:i]
                return new_df
            low = i+1

In [ ]:
df1

,Date,lat,lon
0,2024-12-11 20:36:00,26.958835,-78.510249
1,2024-12-11 20:40:00,15.356222,-68.410381
2,2024-12-11 20:44:00,3.251140,-59.505702
3,2024-12-11 20:48:00,-8.948911,-50.839524
4,2024-12-11 20:52:00,-20.868778,-41.513600
...,...,...,...
85,2024-12-12 02:16:00,18.365540,53.836089
86,2024-12-12 02:20:00,29.782693,64.446255
87,2024-12-12 02:24:00,39.914133,77.839800
88,2024-12-12 02:28:00,47.691038,95.498677


In [ ]:
get_orbit(df1)

,Date,lat,lon
20,2024-12-11 21:56:00,51.499320,-156.953010
21,2024-12-11 22:00:00,47.344195,-134.944162
22,2024-12-11 22:04:00,39.409252,-117.572482
23,2024-12-11 22:08:00,29.203353,-104.392058
24,2024-12-11 22:12:00,17.764327,-93.912555
25,2024-12-11 22:16:00,5.725067,-84.848420
26,2024-12-11 22:20:00,-6.485163,-76.205069
27,2024-12-11 22:24:00,-18.494202,-67.089794
28,2024-12-11 22:28:00,-29.866919,-56.497612
29,2024-12-11 22:32:00,-39.956091,-43.128229


In [ ]:
df2 = obtain_future_date(df, nhours=10)
df2.index = range(len(df1))

In [30]:
df2

,Date,lat,lon
0,2024-12-12 03:52:00,27.543480,38.528773
1,2024-12-12 03:56:00,38.006096,51.235279
2,2024-12-12 04:00:00,46.373514,67.929613
3,2024-12-12 04:04:00,51.187410,89.357664
4,2024-12-12 04:08:00,51.080205,113.109477
...,...,...,...
85,2024-12-12 09:32:00,-51.551146,-154.608124
86,2024-12-12 09:36:00,-47.549930,-132.508791
87,2024-12-12 09:40:00,-39.719119,-114.987658
88,2024-12-12 09:44:00,-29.566527,-101.686394


In [ ]:
get_orbit(df2)

,Date,lat,lon
9,2024-12-12 04:28:00,3.436915,-177.566684
10,2024-12-12 04:32:00,-8.763714,-168.904069
11,2024-12-12 04:36:00,-20.690274,-159.596105
12,2024-12-12 04:40:00,-31.881025,-148.593759
13,2024-12-12 04:44:00,-41.627407,-134.535141
14,2024-12-12 04:48:00,-48.773152,-116.007573
15,2024-12-12 04:52:00,-51.771326,-93.212220
16,2024-12-12 04:56:00,-49.666111,-69.948986
17,2024-12-12 05:00:00,-43.144406,-50.526505
18,2024-12-12 05:04:00,-33.748075,-35.734228


In [28]:
df3 = obtain_future_date(df, nhours=25)
df3.index = range(len(df1))

In [31]:
df3

,Date,lat,lon
0,2024-12-12 19:48:00,27.421042,-72.759053
1,2024-12-12 19:52:00,15.849814,-62.585818
2,2024-12-12 19:56:00,3.757000,-53.651964
3,2024-12-12 20:00:00,-8.445948,-44.993751
4,2024-12-12 20:04:00,-20.384953,-35.714850
...,...,...,...
85,2024-12-13 01:28:00,17.895525,59.666056
86,2024-12-13 01:32:00,29.349456,70.196291
87,2024-12-13 01:36:00,39.549873,83.452424
88,2024-12-13 01:40:00,47.447436,100.923480


In [29]:
get_orbit(df3)

,Date,lat,lon
19,2024-12-12 21:04:00,50.483669,-175.353787
20,2024-12-12 21:08:00,51.567132,-151.713620
21,2024-12-12 21:12:00,47.600105,-129.548077
22,2024-12-12 21:16:00,39.787396,-111.978553
23,2024-12-12 21:20:00,29.650035,-98.655061
24,2024-12-12 21:24:00,18.246717,-88.092750
25,2024-12-12 21:28:00,6.222762,-78.991909
26,2024-12-12 21:32:00,-5.987524,-70.349254
27,2024-12-12 21:36:00,-18.012082,-61.272167
28,2024-12-12 21:40:00,-29.421045,-50.764470
